In [5]:
import os

from sklearn.model_selection import train_test_split
import matplotlib
import argparse
import random
import numpy as np
import csv
import pandas as pd
from datascience import *
import tensorflow as tf
from sklearn.feature_extraction.text import CountVectorizer
from keras.models import Model
from keras.models import Sequential, load_model

from keras.activations import softmax

from keras.layers import *
from keras.optimizers import Adam
from keras import Input
from keras.preprocessing.text import Tokenizer
from keras.utils import plot_model
import keras
#from keras.backend.tensorflow_backend import set_session
#config = tf.ConfigProto()
# config.gpu_options.allocator_type = 'BFC' #A "Best-fit with coalescing" algorithm, simplified from a version of dlmalloc.
# config.gpu_options.per_process_gpu_memory_fraction = 0.7
#config.gpu_options.allow_growth =True

#set_session(tf.Session(config=config)) 
import re
os.environ['PYTHONHASHSEED'] = '0'
random.seed(1)
np.random.seed(1)
tf.set_random_seed(1)
from keras.callbacks import *
from keras.models import model_from_json
import json



Using TensorFlow backend.


In [6]:
def sanitize(string):
    words = string.split(' ')
    return words

In [7]:
!nvidia-smi


Fri Jul 12 07:55:21 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.56       Driver Version: 418.56       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  On   | 00000000:D8:00.0 Off |                  N/A |
| 27%   30C    P8    20W / 250W |   9888MiB / 10989MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [9]:
#read csv

train = pd.read_csv("dataset/Phrase_Level/PAWS_QQP/LABELED_PAWS_QQP_TRAIN.tsv", sep='\t')
dev = pd.read_csv("dataset/Phrase_Level/PAWS_QQP/LABELED_PAWS_QQP_DEV.tsv", sep='\t')


print(train[:5])

train['title1_tokenized'] = \
    train.loc[:, 'sent1'] \
         .apply(sanitize)
train['title2_tokenized'] = \
    train.loc[:, 'sent2'] \
         .apply(sanitize)


dev['title1_tokenized'] = \
    dev.loc[:, 'sent1'] \
         .apply(sanitize)
dev['title2_tokenized'] = \
    dev.loc[:, 'sent2'] \
         .apply(sanitize)


   Unnamed: 0  TRAIN_PAWS_QQP_id  PPDB_id  \
0           0                  4    44326   
1           1                 10   103288   
2           2                 14   316010   
3           3                 18   526060   
4           4                 61    43995   

                                               sent1  \
0  b'What are the most common traffic convictions...   
1  b'Which are in your opinion the most beautiful...   
2            b'Is nuclear energy renewable energy ?'   
3  b'What ancient units would definitely win in a...   
4  b'I have only a very nice startup idea but I m...   

                                               sent2       paraphrase1  \
0  b'What are the most common traffic convictions...      most common    
1  b'Which are in your opinion the most depressin...   most beautiful    
2  b"`` Is nuclear energy `` renewable energy `` ...   nuclear energy    
3  b'What numerous modern units would definitely ...    fight against    
4  b'I have only a ver

In [13]:
train.head()

,Unnamed: 0,TRAIN_PAWS_QQP_id,PPDB_id,sent1,sent2,paraphrase1,paraphrase2,labels1,labels2,Ground Label,title1_tokenized,title2_tokenized
0,0,4,44326,b'What are the most common traffic convictions...,b'What are the most common traffic convictions...,most common,common,O O O B E O O O O O O O O O O O O O O O O O,O O O O S O O O O O O O O O O O O O O O O O,0,"[b'What, are, the, most, common, traffic, conv...","[b'What, are, the, most, common, traffic, conv..."
1,1,10,103288,b'Which are in your opinion the most beautiful...,b'Which are in your opinion the most depressin...,most beautiful,beautiful,O O O O O O B E O O O O O O O O,O O O O O O O O O O O S O O O O,1,"[b'Which, are, in, your, opinion, the, most, b...","[b'Which, are, in, your, opinion, the, most, d..."
2,2,14,316010,b'Is nuclear energy renewable energy ?',"b""`` Is nuclear energy `` renewable energy `` ...",nuclear energy,nuclear,O B E O O O,O O S O O O O O O O,1,"[b'Is, nuclear, energy, renewable, energy, ?']","[b""``, Is, nuclear, energy, ``, renewable, ene..."
3,3,18,526060,b'What ancient units would definitely win in a...,b'What numerous modern units would definitely ...,fight against,fight,O O O O O O O O O B E O O O O,O O O O O O O O O O S O O O O,0,"[b'What, ancient, units, would, definitely, wi...","[b'What, numerous, modern, units, would, defin..."
4,4,61,43995,b'I have only a very nice startup idea but I m...,b'I have only a very good startup idea but I m...,very nice,nice,O O O O B E O O O O O O O O O O O O O,O O O O O O O O O O O O S O O O O O O,1,"[b'I, have, only, a, very, nice, startup, idea...","[b'I, have, only, a, very, good, startup, idea..."


In [11]:
MAX_NUM_WORDS = 10000
tokenizer = keras \
    .preprocessing \
    .text \
    .Tokenizer(num_words=MAX_NUM_WORDS)

train_corpus_x1 = train.title1_tokenized
train_corpus_x2 = train.title2_tokenized
train_corpus = pd.concat([
    train_corpus_x1, train_corpus_x2])
#corpus.shape
tokenizer.fit_on_texts(train_corpus)
tokenizer.texts_to_sequences(train_corpus)
x1_train = tokenizer \
    .texts_to_sequences(train_corpus_x1)
x2_train = tokenizer \
    .texts_to_sequences(train_corpus_x2)


print(tokenizer.document_count)




dev_corpus_x1 = dev.title1_tokenized
dev_corpus_x2 = dev.title2_tokenized
dev_corpus = pd.concat([
    dev_corpus_x1, dev_corpus_x2])
#corpus.shape
tokenizer.fit_on_texts(dev_corpus)
tokenizer.texts_to_sequences(dev_corpus)
x1_dev = tokenizer \
    .texts_to_sequences(dev_corpus_x1)
x2_dev = tokenizer \
    .texts_to_sequences(dev_corpus_x2)




word_index = tokenizer.word_index


print(tokenizer.document_count)

2580
2780


In [6]:
max_seq_len1 = max([
    len(seq) for seq in x1_train])
print(max_seq_len1)

50


In [7]:


MAX_SEQUENCE_LENGTH = 50  #better to have words covered than uncovered
x1_train = keras \
    .preprocessing \
    .sequence \
    .pad_sequences(x1_train, 
                   maxlen=MAX_SEQUENCE_LENGTH)


x2_train = keras \
    .preprocessing \
    .sequence \
    .pad_sequences(x2_train, 
                   maxlen=MAX_SEQUENCE_LENGTH)

x1_dev = keras \
    .preprocessing \
    .sequence \
    .pad_sequences(x1_dev, 
                   maxlen=MAX_SEQUENCE_LENGTH)


x2_dev = keras \
    .preprocessing \
    .sequence \
    .pad_sequences(x2_dev, 
                   maxlen=MAX_SEQUENCE_LENGTH)


x1_test = keras \
    .preprocessing \
    .sequence \
    .pad_sequences(x1_test, 
                   maxlen=MAX_SEQUENCE_LENGTH)

x2_test = keras \
    .preprocessing \
    .sequence \
    .pad_sequences(x2_test, 
                   maxlen=MAX_SEQUENCE_LENGTH)


y_train = train.label

y_train = keras \
    .utils \
    .to_categorical(y_train, num_classes=2)


y_dev = dev.label

y_dev = keras \
    .utils \
    .to_categorical(y_dev, num_classes=2)



y_test = test.label

y_test = keras \
    .utils \
    .to_categorical(y_test, num_classes=2)

x1_test[:3]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,   11,  153,    2,  265,  521,   10,
           2, 1441,  758,    8,    5,   30,   12,  236,  521,  112,    3,
           2, 4981,  758,    6, 4982,    1],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 1850,
         215,    9,  101,  689,  297, 4957,   71,    5,    9,  120,   47,
         108,   28,   54, 1034,   84,   71,   52,   50,    9,  166,  259,
         108,  718,   71,  108, 1037,   71,    5,  853,   71,   20,   59,
          12,    9,  120,   79,  103,   19],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,  122,   12,    9,   96,   26,
          93,  209,  146,   41,  690, 2899,   50,    9,   12,   54,   29,
         752,    3,   26,  185,  146,   34,   81,    7,    2,  690, 2899,
          57, 1030,   

In [14]:
#BiLSTM?
NUM_CLASSES=2 #boolean; 1 or 0

#rough estimate

MAX_SEQUENCE_LENGTH = 50 #how long one sentence is

#this is arbitrary?

NUM_LSTM_UNITS = 128 #output dimension


NUM_EMBEDDING_DIM = 256

top_input = Input(
    shape=(MAX_SEQUENCE_LENGTH, ),    #this is the first sentence
    dtype='int32')

bm_input = Input(
    shape=(MAX_SEQUENCE_LENGTH, ),   #this is the second
    dtype='int32')


embedding_layer = Embedding(
    MAX_NUM_WORDS, NUM_EMBEDDING_DIM)

top_embedded = embedding_layer(
    top_input)
bm_embedded = embedding_layer(
    bm_input)


shared_lstm = Bidirectional(LSTM(NUM_LSTM_UNITS))
top_output = shared_lstm(top_embedded)
bm_output = shared_lstm(bm_embedded)


merged = concatenate(
    [top_output, bm_output], 
    axis=-1)


dense =  Dense(
    units=NUM_CLASSES, 
    activation='sigmoid')
predictions = dense(merged)



model = Model(
    inputs=[top_input, bm_input], 
    outputs=predictions)


model.summary()

model.compile(
    optimizer='Adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'])

BATCH_SIZE = 512

NUM_EPOCHS = 30

earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint('dataset/Phrase_Level/PAWS_QQP_models/PAWS_QQP_TRAIN_PHRASE_LEVEL.h5', save_best_only=True,  verbose=1, monitor='val_loss', mode='min')


Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 50)           0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 50)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 50, 256)      2560000     input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None

In [4]:
#References
#----------
#[1]. Parikh, Ankur P., et al. "A decomposable attention model for natural language inference." arXiv preprint arXiv:1606.01933 (2016).


def StaticEmbedding(embedding_matrix):
    in_dim, out_dim = embedding_matrix.shape
    embedding = Embedding(in_dim, out_dim, weights=[embedding_matrix], trainable=False)
    return embedding

def unchanged_shape(input_shape):
    return input_shape

def time_distributed(x, layers):
    for l in layers:
        x = TimeDistributed(l)(x)
    return x

def align(input_1, input_2):
    attention = Dot(axes=-1)([input_1, input_2])
    w_att_1 = Lambda(lambda x: softmax(x, axis=1),
                     output_shape=unchanged_shape)(attention)
    w_att_2 = Permute((2,1))(Lambda(lambda x: softmax(x, axis=2),
                             output_shape=unchanged_shape)(attention))
    in1_aligned = Dot(axes=1)([w_att_1, input_1])
    in2_aligned = Dot(axes=1)([w_att_2, input_2])
    return in1_aligned, in2_aligned

def aggregate(x1, x2, num_class, dense_dim=300, dropout_rate=0.2, activation="relu"):
    feat1 = concatenate(map(lambda l: l(x1), [GlobalAvgPool1D(), GlobalMaxPool1D()]))
    feat2 = concatenate(map(lambda l: l(x2), [GlobalAvgPool1D(), GlobalMaxPool1D()]))
    x = Concatenate()([feat1, feat2])
    x = BatchNormalization()(x)
    x = Dense(dense_dim, activation=activation)(x)
    x = Dropout(dropout_rate)(x)
    x = BatchNormalization()(x)
    x = Dense(dense_dim, activation=activation)(x)
    x = Dropout(dropout_rate)(x)
    scores = Dense(num_class, activation='sigmoid')(x)
    return scores    

def build_model(embedding_matrix, num_class=1, 
                           projection_dim=300, projection_hidden=0, projection_dropout=0.2,
                           compare_dim=500, compare_dropout=0.2,
                           dense_dim=300, dropout_rate=0.2,
                           lr=1e-3, activation='relu', maxlen=30):
    q1 = Input(name='q1',shape=(maxlen,))
    q2 = Input(name='q2',shape=(maxlen,))
    
    # Embedding
    encode = StaticEmbedding(embedding_matrix)
    q1_embed = encode(q1)
    q2_embed = encode(q2)
    
    # Projection
    projection_layers = []
    if projection_hidden > 0:
        projection_layers.extend([
                Dense(projection_hidden, activation=activation),
                Dropout(rate=projection_dropout),
            ])
    projection_layers.extend([
            Dense(projection_dim, activation=None),
            Dropout(rate=projection_dropout),
        ])
    q1_encoded = time_distributed(q1_embed, projection_layers)
    q2_encoded = time_distributed(q2_embed, projection_layers)
    
    # Attention
    q1_aligned, q2_aligned = align(q1_encoded, q2_encoded)    
    
    # Compare
    q1_combined = concatenate([q1_encoded, q2_aligned])
    q2_combined = concatenate([q2_encoded, q1_aligned]) 
    compare_layers = [
        Dense(compare_dim, activation=activation),
        Dropout(compare_dropout),
        Dense(compare_dim, activation=activation),
        Dropout(compare_dropout),
    ]
    q1_compare = time_distributed(q1_combined, compare_layers)
    q2_compare = time_distributed(q2_combined, compare_layers)
    
    # Aggregate
    scores = aggregate(q1_compare, q2_compare, num_class)
    
    model = Model(inputs=[q1, q2], outputs=scores)
    return model

if __name__ == "__main__":
    import numpy as np
    model = build_model(embedding_matrix=np.zeros((30, 20)), projection_hidden=200)
    model.summary() 


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


ValueError: Layer concatenate_3 was called with an input that isn't a symbolic tensor. Received type: <class 'map'>. Full input: [<map object at 0x7fdd0bb78f28>]. All inputs to the layer should be tensors.

In [11]:
history = model.fit(x=[x1_train, x2_train], y=y_train, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS, callbacks=[mcp_save], validation_data=([x1_dev, x2_dev], y_dev),
    
    shuffle=True
)

Instructions for updating:
Use tf.cast instead.
Train on 11863 samples, validate on 669 samples
Epoch 1/30


KeyboardInterrupt: 

In [47]:

model.save("BiLSTM_PAWS_QQP.h5")        ##first attempt
print("Saved model to disk")  

Saved model to disk


In [12]:
model = load_model("BiLSTM_PAWS_QQP.h5")


In [13]:

predicts = model.predict(
    [x1_train, x2_train])

In [14]:
print(len(predicts))

11863


In [15]:
real1=train.sentence1
real2=train.sentence2
realCat=train.label
no = []
err1 = []
err2 = []
wrongCat = []


for x in range(len(predicts)):
    if (np.argmax(predicts[x])!=(train.label[x])):
        no=np.append(no, x)
        err1=np.append(err1, real1[x])
        err2=np.append(err2, real2[x])
        wrongCat = np.append(wrongCat, realCat[x])
print(1-(len(wrongCat)/len(predicts)))

0.7814212256596139


In [16]:
error_analysis = Table()


error_analysis = error_analysis.with_column("id", no, "sent1", err1, "sent2", err2, "label", wrongCat)


#false POSITIVE: swap two words
#false negative: swap two specific nouns
error_analysis.show(30)

id,sent1,sent2,label
1,b'How can you treat ocd ? Is there any helpful suggestio ...,b'How can you treat OCD ? Is there any helpful suggestio ...,1
7,b'What are some beautiful lines to comment on Beautiful ...,b'What are some Beautiful lines to comment on beautiful ...,1
8,b'Can small dogs breed with large dogs ?',b'Can small dogs breed with large dogs ?',1
13,b'Are Indian IT services companies like Tech Mahindra/ W ...,b'Are Indian IT services companies like Wipro/TCS/Tech . ...,1
14,b'Is nuclear energy renewable energy ?',"b""`` Is nuclear energy `` renewable energy `` ? ''""",1
15,b'What should I solve and what or how should I revise fo ...,b'What should I revise and what or how should I solve fo ...,0
20,b'How is the relative ratio of brain waves ( alpha/beta/ ...,b'How is the ratio of relative brain waves ( alpha/beta/ ...,0
21,b'How can I make an android app using Python 3 ? Is ther ...,b'How can I develop an android app using Python 3 ? Is t ...,1
24,b'Headbreak : What can I do to prevent heart break night ...,b'Headbreak : What can I do to prevent heart break night ...,1
25,b'When is latent heat positive ?',b'When is latent heat positive ?',1


id,sent1,sent2,label
1,b'How can you treat ocd ? Is there any helpful suggestio ...,b'How can you treat OCD ? Is there any helpful suggestio ...,1
7,b'What are some beautiful lines to comment on Beautiful ...,b'What are some Beautiful lines to comment on beautiful ...,1
8,b'Can small dogs breed with large dogs ?',b'Can small dogs breed with large dogs ?',1
13,b'Are Indian IT services companies like Tech Mahindra/ W ...,b'Are Indian IT services companies like Wipro/TCS/Tech . ...,1
14,b'Is nuclear energy renewable energy ?',"b""`` Is nuclear energy `` renewable energy `` ? ''""",1
21,b'How can I make an android app using Python 3 ? Is ther ...,b'How can I develop an android app using Python 3 ? Is t ...,1
24,b'Headbreak : What can I do to prevent heart break night ...,b'Headbreak : What can I do to prevent heart break night ...,1
25,b'When is latent heat positive ?',b'When is latent heat positive ?',1
28,"b""What does the phrase 'Do you feel me ' mean ?""","b""What does the phrase `Do you feel me'mean ?""",1
29,b'What are some unexpected things first-time visitors to ...,"b'What are some unexpected things , first-time visitors ...",1
